In [2]:
import pandas as pd
import numpy as np

In [3]:
%%time
#read data and drop bad rows
df = pd.read_csv("./news_dataset.csv")
df.replace(r'^\s*$', np.nan, regex=True,inplace=True)
df.replace('', np.nan, inplace=True)
print("datasize: ",len(df))
df.dropna(subset=['content'], inplace=True)
print("Drop empty content...")
print("new datasize: ",len(df))

ColumnName = "content_new"
df[ColumnName] = df.content.copy()
df[ColumnName] = df[ColumnName].str.lower()
df[ColumnName] = df[ColumnName].str.strip()
df.head()

# %%time
#remove stopwords in content for each row, this is done in place so df['content'] is also modifed...

def removeStopwords(text):
    from gensim.parsing.preprocessing import remove_stopwords
    # for element in textCol:
    return remove_stopwords(text)

def removeStopwordsInDF(df,ColumnName = "text"):
    from gensim.parsing.preprocessing import remove_stopwords
    df[ColumnName] = df[ColumnName].apply(removeStopwords)
    # for element in textCol:
    return df
def striplist(l):
    return([x.strip() for x in l])
df = removeStopwordsInDF(df,ColumnName = "content_new")
# print("output",df_new.content[1])


datasize:  28711
Drop empty content...
new datasize:  28557
CPU times: user 8.39 s, sys: 935 ms, total: 9.33 s
Wall time: 10.5 s


In [4]:
df.head(100)


,Unnamed: 0,title,content,publication,label,content_new
0,0,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,100percentfedup,fake,print pay money plus interest. entire family c...
1,1,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,100percentfedup,fake,attorney general loretta lynch plead fifth? ba...
2,2,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,100percentfedup,fake,red state : fox news sunday reported morning a...
3,3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,100percentfedup,fake,email kayla mueller prisoner tortured isis cha...
4,4,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,100percentfedup,fake,email healthcare reform america great march 20...
5,5,Hillary Goes Absolutely Berserk On Protester A...,Print Hillary goes absolutely berserk! She exp...,100percentfedup,fake,print hillary goes absolutely berserk! explode...
6,6,BREAKING! NYPD Ready To Make Arrests In Weiner...,BREAKING! NYPD Ready To Make Arrests In Weiner...,100percentfedup,fake,breaking! nypd ready arrests weiner case…hilla...
7,7,WOW! WHISTLEBLOWER TELLS CHILLING STORY Of Mas...,BREAKING! NYPD Ready To Make Arrests In Weiner...,100percentfedup,fake,breaking! nypd ready arrests weiner case…hilla...
8,8,BREAKING: CLINTON CLEARED...Was This A Coordin...,\nLimbaugh said that the revelations in the Wi...,100percentfedup,fake,limbaugh said revelations wikileaks material “...
9,9,"EVIL HILLARY SUPPORTERS Yell ""F*ck Trump""…Burn...",Email \nThese people are sick and evil. They w...,100percentfedup,fake,email people sick evil. stop way. laws mean th...


In [5]:
#########################
#DO NOT RUN THE BELOW CELLS. YOU WILL REGRET IF YOU DO SO
#########################

In [9]:
%%time
import spacy
nlp = spacy.load('en')

def process_txt(token):
    if token.is_punct or token.is_space or token.is_stop or token.like_num or token.is_left_punct \
    or token.is_right_punct or token.is_bracket or token.is_quote or token.is_currency or \
    token.like_url or token.like_email:
        return True
    else:
        return False

    
def process_char(token):
    return token.is_punct or token.is_left_punct or token.is_right_punct or \
    token.is_bracket or token.is_quote or token.is_currency


def lemmatize_char(arr):
    array = list(arr)
    lemma = []
    for text in nlp.pipe(array, batch_size=5, n_threads=100):
        for n in text:
            if not process_char(n):
                temp = ''.join(n.lemma_)
                lemma.append(temp)
            else:
                lemma.append(' ')
    return ''.join(lemma)


def lemmatize_txt(array):
    lemma = []
    for text in nlp.pipe(array, batch_size=5, n_threads=100):
        templemma = []
        if text.is_parsed:
            for n in text:
                if not process_txt(n):
                    if n.lemma_ != '-PRON-':
                        n.lemma_ = lemmatize_char(n.lemma_)
                        temp = ''.join(n.lemma_)
                        templemma.append(temp)
        else:
            templemma.append(None)
        lemma.append(templemma)
    return lemma


def calllem(inpu):
    lstele = []
    lstele.append(inpu)
    result = lemmatize_txt(lstele)
    concat_art = ' '.join(result[0])
    inpu = concat_art[:]
    return inpu

nlp.vocab[";"].is_punct = True
nlp.vocab["$"].is_punct = True
df = df.dropna(subset=['content_new'])


for i in range(0, df['content_new'].size+1, 250):
    temparr =  df['content_new'][i:i+250]
    near = list(map(calllem, temparr))
    df['content_new'][i:i+250] = near
    
    if i%1000 == 0:
        print("Finish first ", i, " rows")



/Users/yu-hsiangchou/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Finish first  0  rows
Finish first  1000  rows
Finish first  2000  rows
Finish first  3000  rows
Finish first  4000  rows
Finish first  5000  rows
Finish first  6000  rows
Finish first  7000  rows
Finish first  8000  rows
Finish first  9000  rows
Finish first  10000  rows
Finish first  11000  rows
Finish first  12000  rows
Finish first  13000  rows
Finish first  14000  rows
Finish first  15000  rows
Finish first  16000  rows
Finish first  17000  rows
Finish first  18000  rows
Finish first  19000  rows
Finish first  20000  rows
Finish first  21000  rows
Finish first  22000  rows
Finish first  23000  rows
Finish first  24000  rows
Finish first  25000  rows
Finish first  26000  rows
Finish first  27000  rows
Finish first  28000  rows
CPU times: user 2d 21h 14min 29s, sys: 40min 45s, total: 2d 21h 55min 15s
Wall time: 1d 12h 22min 3s


In [13]:
df.head(100)

,Unnamed: 0,title,content,publication,label,content_new
0,0,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,100percentfedup,fake,print pay money plus interest entire family co...
1,1,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,100percentfedup,fake,attorney general loretta lynch plead fifth bar...
2,2,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,100percentfedup,fake,red state fox news sunday report morning antho...
3,3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,100percentfedup,fake,email kayla muell prisoner torture isis chance...
4,4,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,100percentfedup,fake,email healthcare reform america great march am...
5,5,Hillary Goes Absolutely Berserk On Protester A...,Print Hillary goes absolutely berserk! She exp...,100percentfedup,fake,print hillary go absolutely berserk explode ra...
6,6,BREAKING! NYPD Ready To Make Arrests In Weiner...,BREAKING! NYPD Ready To Make Arrests In Weiner...,100percentfedup,fake,break nypd ready arrest wein case hillary visi...
7,7,WOW! WHISTLEBLOWER TELLS CHILLING STORY Of Mas...,BREAKING! NYPD Ready To Make Arrests In Weiner...,100percentfedup,fake,break nypd ready arrest wein case hillary visi...
8,8,BREAKING: CLINTON CLEARED...Was This A Coordin...,\nLimbaugh said that the revelations in the Wi...,100percentfedup,fake,limbaugh say revelation wikileak material star...
9,9,"EVIL HILLARY SUPPORTERS Yell ""F*ck Trump""…Burn...",Email \nThese people are sick and evil. They w...,100percentfedup,fake,email people sick evil stop way law mean mean ...


In [14]:
#export new df to csv file
file_name = "./news_dataset_lemmatized.csv"
df.to_csv(file_name, encoding='utf-8', index=False)

In [3]:
###################
#do not run the above cells
#run this cell to get the new df
###################
df = pd.read_csv("./news_dataset_lemmatized.csv")
df.head(100)

,Unnamed: 0,title,content,publication,label,content_new
0,0,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,100percentfedup,fake,print pay money plus interest entire family co...
1,1,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,100percentfedup,fake,attorney general loretta lynch plead fifth bar...
2,2,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,100percentfedup,fake,red state fox news sunday report morning antho...
3,3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,100percentfedup,fake,email kayla muell prisoner torture isis chance...
4,4,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,100percentfedup,fake,email healthcare reform america great march am...
5,5,Hillary Goes Absolutely Berserk On Protester A...,Print Hillary goes absolutely berserk! She exp...,100percentfedup,fake,print hillary go absolutely berserk explode ra...
6,6,BREAKING! NYPD Ready To Make Arrests In Weiner...,BREAKING! NYPD Ready To Make Arrests In Weiner...,100percentfedup,fake,break nypd ready arrest wein case hillary visi...
7,7,WOW! WHISTLEBLOWER TELLS CHILLING STORY Of Mas...,BREAKING! NYPD Ready To Make Arrests In Weiner...,100percentfedup,fake,break nypd ready arrest wein case hillary visi...
8,8,BREAKING: CLINTON CLEARED...Was This A Coordin...,\nLimbaugh said that the revelations in the Wi...,100percentfedup,fake,limbaugh say revelation wikileak material star...
9,9,"EVIL HILLARY SUPPORTERS Yell ""F*ck Trump""…Burn...",Email \nThese people are sick and evil. They w...,100percentfedup,fake,email people sick evil stop way law mean mean ...


In [7]:
df.dropna(subset=['title'], inplace=True)
ColumnName = "title_new"
df[ColumnName] = df.title.copy()
df.head()

,Unnamed: 0,title,content,publication,label,content_new,title_new
0,0,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,100percentfedup,fake,print pay money plus interest entire family co...,Muslims BUSTED: They Stole Millions In Gov’t B...
1,1,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,100percentfedup,fake,attorney general loretta lynch plead fifth bar...,Re: Why Did Attorney General Loretta Lynch Ple...
2,2,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,100percentfedup,fake,red state fox news sunday report morning antho...,BREAKING: Weiner Cooperating With FBI On Hilla...
3,3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,100percentfedup,fake,email kayla muell prisoner torture isis chance...,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...
4,4,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,100percentfedup,fake,email healthcare reform america great march am...,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...


In [8]:
%%time
import spacy
nlp = spacy.load('en')

def process_txt(token):
    if token.is_punct or token.is_space or token.is_stop or token.like_num or token.is_left_punct \
    or token.is_right_punct or token.is_bracket or token.is_quote or token.is_currency or \
    token.like_url or token.like_email:
        return True
    else:
        return False

    
def process_char(token):
    return token.is_punct or token.is_left_punct or token.is_right_punct or \
    token.is_bracket or token.is_quote or token.is_currency


def lemmatize_char(arr):
    array = list(arr)
    lemma = []
    for text in nlp.pipe(array, batch_size=5, n_threads=100):
        for n in text:
            if not process_char(n):
                temp = ''.join(n.lemma_)
                lemma.append(temp)
            else:
                lemma.append(' ')
    return ''.join(lemma)


def lemmatize_txt(array):
    lemma = []
    for text in nlp.pipe(array, batch_size=5, n_threads=100):
        templemma = []
        if text.is_parsed:
            for n in text:
                if not process_txt(n):
                    if n.lemma_ != '-PRON-':
                        n.lemma_ = lemmatize_char(n.lemma_)
                        temp = ''.join(n.lemma_)
                        templemma.append(temp)
        else:
            templemma.append(None)
        lemma.append(templemma)
    return lemma


def calllem(inpu):
    lstele = []
    lstele.append(inpu)
    result = lemmatize_txt(lstele)
    concat_art = ' '.join(result[0])
    inpu = concat_art[:]
    return inpu

nlp.vocab[";"].is_punct = True
nlp.vocab["$"].is_punct = True
df = df.dropna(subset=['title_new'])


for i in range(0, df['title_new'].size+1, 250):
    temparr =  df['title_new'][i:i+250]
    near = list(map(calllem, temparr))
    df['title_new'][i:i+250] = near
    
    if i%1000 == 0:
        print("Finish first ", i, " rows")

/Users/yu-hsiangchou/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Finish first  0  rows
Finish first  1000  rows
Finish first  2000  rows
Finish first  3000  rows
Finish first  4000  rows
Finish first  5000  rows
Finish first  6000  rows
Finish first  7000  rows
Finish first  8000  rows
Finish first  9000  rows
Finish first  10000  rows
Finish first  11000  rows
Finish first  12000  rows
Finish first  13000  rows
Finish first  14000  rows
Finish first  15000  rows
Finish first  16000  rows
Finish first  17000  rows
Finish first  18000  rows
Finish first  19000  rows
Finish first  20000  rows
Finish first  21000  rows
Finish first  22000  rows
Finish first  23000  rows
Finish first  24000  rows
Finish first  25000  rows
Finish first  26000  rows
Finish first  27000  rows
CPU times: user 1h 22min 31s, sys: 54.5 s, total: 1h 23min 26s
Wall time: 44min


In [9]:
df.head(100)

,Unnamed: 0,title,content,publication,label,content_new,title_new
0,0,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,100percentfedup,fake,print pay money plus interest entire family co...,muslims busted steal millions in gov t benefits
1,1,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,100percentfedup,fake,attorney general loretta lynch plead fifth bar...,re why do attorney general loretta lynch plead...
2,2,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,100percentfedup,fake,red state fox news sunday report morning antho...,break weiner cooperate with fbi on hillary ema...
3,3,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,100percentfedup,fake,email kayla muell prisoner torture isis chance...,pin drop speech by father of daughter kidnappe...
4,4,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,100percentfedup,fake,email healthcare reform america great march am...,fantastic trump s point plan to reform health...
5,5,Hillary Goes Absolutely Berserk On Protester A...,Print Hillary goes absolutely berserk! She exp...,100percentfedup,fake,print hillary go absolutely berserk explode ra...,hillary go absolutely berserk on protester at ...
6,6,BREAKING! NYPD Ready To Make Arrests In Weiner...,BREAKING! NYPD Ready To Make Arrests In Weiner...,100percentfedup,fake,break nypd ready arrest wein case hillary visi...,break nypd ready to make arrests in weiner cas...
7,7,WOW! WHISTLEBLOWER TELLS CHILLING STORY Of Mas...,BREAKING! NYPD Ready To Make Arrests In Weiner...,100percentfedup,fake,break nypd ready arrest wein case hillary visi...,wow whistleblower tell chill story of massive ...
8,8,BREAKING: CLINTON CLEARED...Was This A Coordin...,\nLimbaugh said that the revelations in the Wi...,100percentfedup,fake,limbaugh say revelation wikileak material star...,break clinton cleared be this a coordinated la...
9,9,"EVIL HILLARY SUPPORTERS Yell ""F*ck Trump""…Burn...",Email \nThese people are sick and evil. They w...,100percentfedup,fake,email people sick evil stop way law mean mean ...,evil hillary supporter yell f ck trump burn t...


In [10]:
#export new df to csv file
file_name = "./news_dataset_lemmatized_new.csv"
df.to_csv(file_name, encoding='utf-8', index=False)